In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy import create_engine, select, and_
from sqlalchemy.orm import sessionmaker


In [6]:
# Base class for declarative models
Base = declarative_base()

# Define the models
class SqFeetData(Base):
    __tablename__ = 'sqfeet_data'
    state = Column(String, primary_key=True)
    median_home_price = Column(Integer)
    median_home_size = Column(Integer)
    income_data = relationship("IncomeData", uselist=False, back_populates="sqfeet_data")
    cities_data = relationship("CitiesData", uselist=False, back_populates="sqfeet_data")

class IncomeData(Base):
    __tablename__ = 'income_data'
    state = Column(String, primary_key=True)
    earners_1 = Column(Integer)
    earners_2 = Column(Integer)
    earners_3 = Column(Integer)
    earners_4 = Column(Integer)
    sqfeet_data_state = Column(String, ForeignKey('sqfeet_data.state'))
    sqfeet_data = relationship("SqFeetData", back_populates="income_data")

class CitiesData(Base):
    __tablename__ = 'cities_data'
    state = Column(String, primary_key=True)
    region = Column(String)
    acronyms = Column(String)
    price = Column(Integer)
    sqfeet_data_state = Column(String, ForeignKey('sqfeet_data.state'))
    sqfeet_data = relationship("SqFeetData", back_populates="cities_data")



In [10]:
# SQLite database engine
engine = create_engine('sqlite:///Resources/project3.sqlite')

# Create the tables
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Example of an INNER JOIN query
results = session.query(SqFeetData, CitiesData, IncomeData).\
    filter(SqFeetData.state == CitiesData.state).\
    filter(SqFeetData.state == IncomeData.state).\
    all()

# Print the results
for sqfeet, city, income in results:
    print(f"State: {sqfeet.state}, Median Home Price: {sqfeet.median_home_price}, Region: {city.region}, Earners 1: {income.earners_1}")

# Close the session
session.close()

OperationalError: (sqlite3.OperationalError) no such column: sqfeet_data.median_home_price
[SQL: SELECT sqfeet_data.state AS sqfeet_data_state, sqfeet_data.median_home_price AS sqfeet_data_median_home_price, sqfeet_data.median_home_size AS sqfeet_data_median_home_size, cities_data.state AS cities_data_state, cities_data.region AS cities_data_region, cities_data.acronyms AS cities_data_acronyms, cities_data.price AS cities_data_price, cities_data.sqfeet_data_state AS cities_data_sqfeet_data_state, income_data.state AS income_data_state, income_data.earners_1 AS income_data_earners_1, income_data.earners_2 AS income_data_earners_2, income_data.earners_3 AS income_data_earners_3, income_data.earners_4 AS income_data_earners_4, income_data.sqfeet_data_state AS income_data_sqfeet_data_state 
FROM sqfeet_data, cities_data, income_data 
WHERE sqfeet_data.state = cities_data.state AND sqfeet_data.state = income_data.state]
(Background on this error at: https://sqlalche.me/e/14/e3q8)